In [1]:
import pandas as pd
import pymongo
from pymongo import MongoClient
from pandas.io.json import json_normalize
import math
from numpy import nan
import numpy as np

In [2]:
from sshtunnel import SSHTunnelForwarder

server = SSHTunnelForwarder(
    '128.226.28.123',
    ssh_username="rthakur1",
    ssh_password="XOfoxPi24",
    remote_bind_address=('127.0.0.1', 27017)
)

server.start()

print(server.local_bind_port)  # show assigned local port
# work with `SECRET SERVICE` through `server.local_bind_port`.

#server.stop()

63001


In [3]:
client=pymongo.MongoClient('127.0.0.1',server.local_bind_port)
mydb=client['SMDP']
tmdb_collection=mydb['tmdb_tv_shows']

In [4]:
print(tmdb_collection)
print(type(tmdb_collection))

Collection(Database(MongoClient(host=['127.0.0.1:63001'], document_class=dict, tz_aware=False, connect=True), 'SMDP'), 'tmdb_tv_shows')
<class 'pymongo.collection.Collection'>


In [5]:

data_to_predict=list(tmdb_collection.find({"first_air_date":{"$gt": "2022-30-12"}}))
dataframe_to_predict = json_normalize(data_to_predict)
dataframe_to_predict.to_pickle("tmdb_data_prediction.pkl")
df_tmdb_prediction=pd.read_pickle("tmdb_data_prediction.pkl")


In [6]:
df_tmdb_prediction

,_id,backdrop_path,first_air_date,genre_ids,id,name,origin_country,original_language,original_name,overview,...,show_details.last_episode_to_air.production_code,show_details.last_episode_to_air.runtime,show_details.last_episode_to_air.season_number,show_details.last_episode_to_air.show_id,show_details.last_episode_to_air.still_path,show_details.last_episode_to_air.vote_average,show_details.last_episode_to_air.vote_count,credits.success,credits.status_code,credits.status_message
0,63614572e482f08429a7fc98,None,2024-12-12,[35],213569,Western,[US],en,Western,"Set in the 1800s, it follows Polly, a young hi...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,63614572e482f08429a7fc99,None,2024-12-12,[],213338,The Buccaneers,[US],en,The Buccaneers,The Buccaneers adaptation follows a group of f...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,63614572e482f08429a7fc9a,None,2024-12-12,[],213337,Still Up,[GB],en,Still Up,"Still Up follows the exploits of Lisa, an impu...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,63614572e482f08429a7fc9b,None,2024-12-12,[],212762,The Gallows Pole,[GB],en,The Gallows Pole,Set against the backdrop of the coming industr...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,63614572e482f08429a7fc9c,None,2024-12-12,[],212462,Mary & George,[US],en,Mary & George,"The story about Mary Villiers, Countess of Buc...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,63614572e482f08429a7fcfa,None,2023-01-01,"[16, 35]",203203,Married... with Children,[US],en,Married... with Children,An animated adaptation of the hit 1980s sitcom...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,63614572e482f08429a7fcfb,None,2023-01-01,[35],202738,Greenlight – German Genius,[DE],de,Greenlight – German Genius,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99,63614573e482f08429a7fcfc,None,2023-01-01,[],195257,15 Candles,[US],en,15 Candles,Four young Latinas in high school navigate fee...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,63614573e482f08429a7fcfd,None,2023-01-01,[],139167,Night Wanderer,[],zh,夜旅人,"One summer night in Shanghai, Zong Ying bumps ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
rows=df_tmdb_prediction.shape[0]
print(rows)

102


## Preprocessing the data

### 1.Eliminating the unrequired columns and storing just the required ones for prediction

In [8]:
def required_columns(df_tmdb_prediction):
    df = df_tmdb_prediction[['id','show_details.name','first_air_date','credits.cast','show_details.genres','show_details.number_of_episodes','show_details.number_of_seasons','show_details.overview','show_details.production_companies','show_details.status','show_details.type','show_details.episode_run_time']]

    #renaming the columns
    df.rename(columns={'show_details.name':'name','credits.cast':'cast_details','first_air_date':'first_air_date','show_details.genres': 'genre', 'show_details.number_of_episodes': 'number_of_episodes','show_details.overview':'overview','show_details.production_companies':'production_companies','show_details.episode_run_time':'episode_run_time','show_details.number_of_seasons':'number_of_seasons','show_details.type':'type','show_details.status':'status'}, inplace=True)
    
    
    df=df[['id','name','first_air_date','genre','cast_details','episode_run_time','number_of_episodes','number_of_seasons','overview','production_companies','status','type']]
    #df.head(10)
    return df

### 2. Preprocessing Genre Columns

In [9]:
def preprocessing_genre(df):
    genre_type_list=['Comedy', 'Documentary', 'Animation', 'Drama', 'Mystery', 'Crime', 'Action & Adventure', 'Sci-Fi & Fantasy', 'Reality', 'Talk', 'News', 'Kids', 'Family', 'Soap', 'Western', 'War & Politics', 'History', 'Music', 'Romance', 'Erotic']
    df['genre'] = df['genre'].fillna(0)
           
    for r in range(0,rows):
        if df['genre'][r]==0:
            df['genre'][r]=[]
        else:
            genre_ls=[]

            for g in df['genre'][r]:
                genre_ls.append(g['name'])


            if len(genre_ls)>0:
                df['genre'][r]=genre_ls
            else:
                df['genre'][r]=[]

    
        
    #One hot Encoding Genre Columns
    
    mlb = MultiLabelBinarizer()
    df
    genre_df=pd.concat([
        pd.DataFrame(mlb.fit_transform(df['genre']), columns=mlb.classes_),
    ], 1)

    df=df.join(genre_df)
    #df.head(10)
    
    return df
         


### 3.Preprocessing Status and Type Columns

In [10]:
def preprocessing_status_type(df):
    df = pd.get_dummies(df, columns = ['type','status'])
    df.head(10)
    return df
    
    

### 4.Preprocessing Production Companies

In [11]:
final_prod_names=['NBC', 'BBC', 'Warner Bros. Television', 'Studio Dragon', 'Estúdios Globo', 'Universal Television', 'Amazon Studios', 'TVB', 'CBS Studios', 'TV Tokyo', 'Madhouse', 'NHK', 'Toei Animation', 'Sony Pictures Television Studios', 'SUNRISE', 'Pol-Ka Producciones', 'Pink Pineapple', '20th Century Fox Television', 'Toei Company', 'TV Asahi']
def preprocessing_production_companies(df):
    df['production_companies'] = df['production_companies'].fillna(0)
    
    #For extracting production_companies in each row and omitting out the ids

    for r in range(0,rows):
        if df['production_companies'][r]==0:
            df['production_companies'][r]=[]
        else:
            prod_ls=[]

            for p in df['production_companies'][r]:
                prod_ls.append(p['name'])


            if len(prod_ls)>0:
                df['production_companies'][r]=prod_ls
            else:
                df['production_companies'][r]=[]

            #print(prod_ls)
            
    
    #Keeping just top 20 prod names in each row

    for r in range(0,rows):
        if df['production_companies'][r]==[]:
            continue
        else:
            new_prod_list=[]
            for company in df['production_companies'][r]:
                if company in final_prod_names:
                    new_prod_list.append(company)
            if len(new_prod_list)>0:
                df['production_companies'][r]=new_prod_list
            else:
                df['production_companies'][r]=[]
            #print("row added for ",r)

    
    #One Hot encoded Produciton comp
    from sklearn.preprocessing import MultiLabelBinarizer
    mlb = MultiLabelBinarizer()
    prod_company_df=pd.concat([
        pd.DataFrame(mlb.fit_transform(df['production_companies']), columns=mlb.classes_),
    ], 1)
    df=df.join(prod_company_df)
    #df.pop('production_companies')
    ## Now we have 64 columns
    
    df.head(10)
    
    return df
    

### 5.Preprocessing Episode Run Time

In [12]:
from statistics import mean
def preprocessing_episode_runtime(df):
    
    df['episode_run_time'] = df['episode_run_time'].fillna(0)
    for r in range(0,rows):
        if df['episode_run_time'][r]==[] or df['episode_run_time'][r]==0:
            df['episode_run_time'][r]=0
        else:

            if len(df['episode_run_time'][r])==1:            
                df['episode_run_time'][r]=df['episode_run_time'][r][0]
            else:

                df['episode_run_time'][r]=mean(df['episode_run_time'][r])
            #print("added for row",r)
    
    if (df['episode_run_time'].isnull().sum()==0):
        df.head(10)
    else:
        print("Error in processing episode runtime column")
    return df


### 6.Preprocessing Cast Details

In [13]:

def preprocessing_cast_details(df):
    
    df['cast_details'] = df['cast_details'].fillna(0)
    #adding TV cast names and TV cast popularity in each row
    
    tv_cast_name=[]
    tv_cast_popularity=[]
    for r in range(0,rows):
        cast_name_list=[]
        popularity_list=[]
        final_list=[]
        if df['cast_details'][r]==[] or df['cast_details'][r]==0 or df['cast_details'][r]==None :
            df['cast_details'][r]=None
            tv_cast_name.append([])
            tv_cast_popularity.append(0)
            continue
        else:
            for each_cast in df['cast_details'][r]:
                cast_name_list.append(each_cast['name'])
                popularity_list.append(each_cast['popularity'])

            mean_cast_popularity=mean(popularity_list)
        
        tv_cast_name.append(cast_name_list)
        tv_cast_popularity.append(mean_cast_popularity)
    
    df['Casts']=tv_cast_name
    df['Casts_avg_popularity']=tv_cast_popularity
    
    
               
    return df

### 7.Preprocessing Default Air Date

In [14]:
def preprocessing_defualt_air_date(df):
    df['first_air_date']=df['first_air_date'].replace(np.nan, '2020-12-12')
    df.head(10)
    
    return df

### 8.Preprocessing Genre Names

In [15]:
def changing_genre_names(df):
    genre_type_list=['Comedy', 'Documentary', 'Animation', 'Drama', 'Mystery', 'Crime', 'Action & Adventure', 'Sci-Fi & Fantasy', 'Reality', 'Talk', 'News', 'Kids', 'Family', 'Soap', 'Western', 'War & Politics', 'History', 'Music', 'Romance', 'Erotic']
    for x in range(len(genre_type_list)):
        og_column_name=genre_type_list[x]
        new_col_name="Genre_"+genre_type_list[x]
        df.rename(columns = {og_column_name: new_col_name}, 
              inplace = True)
    
    return df
    

### 9.Popping Columns

In [16]:
def popping_columns(df):
    df.pop('genre')
    df.pop('cast_details')
    df.pop('overview')
    df.pop('first_air_date')
    #df.pop('name')
    df.pop('id')
    
    return df

### 10.Changing Prod Company Names

In [17]:
def changing_prod_companynames(df):
    for x in range(len(final_prod_names)):
        og_column_name=final_prod_names[x]
        new_col_name="ProdComp_"+final_prod_names[x]
        df.rename(columns = {og_column_name: new_col_name}, 
              inplace = True)
    
    return df

### 11. Normalizing Values

In [18]:

def replacing_nan_values_normalizing(df):
    def absolute_maximum_scale(series):
        return series / series.abs().max()

    df['number_of_episodes']=df['number_of_episodes'].fillna(df['number_of_episodes'].mean())
    df['number_of_seasons']=df['number_of_seasons'].fillna(df['number_of_seasons'].mean())
    
    #normalizing
    df['episode_run_time'] = absolute_maximum_scale(df['episode_run_time'])
    df['number_of_episodes'] = absolute_maximum_scale(df['number_of_episodes'])
    df['number_of_seasons'] = absolute_maximum_scale(df['number_of_seasons'])
    df['Casts_avg_popularity'] = absolute_maximum_scale(df['Casts_avg_popularity'])
    
    return df
    
   
    
    

### 12. Popping Out Further unrequired columns

In [19]:
def popping_columns_final(df):
    df.pop('Casts')
    df.pop('production_companies')
    return df

###  Importing the trained model  from pickle

In [20]:
import pickle
pickled_model = pickle.load(open('model.pkl', 'rb'))

In [21]:
df=required_columns(df_tmdb_prediction)


In [22]:
df

,id,name,first_air_date,genre,cast_details,episode_run_time,number_of_episodes,number_of_seasons,overview,production_companies,status,type
0,213569,Western,2024-12-12,"[{'id': 35, 'name': 'Comedy'}]","[{'adult': False, 'gender': 1, 'id': 1489288, ...",[],1,1,"Set in the 1800s, it follows Polly, a young hi...","[{'id': 11073, 'logo_path': '/aCbASRcI1MI7DXjP...",Planned,Scripted
1,213338,The Buccaneers,2024-12-12,[],"[{'adult': False, 'gender': 1, 'id': 110014, '...",[],1,1,The Buccaneers adaptation follows a group of f...,[],In Production,Scripted
2,213337,Still Up,2024-12-12,[],"[{'adult': False, 'gender': 1, 'id': 221981, '...",[],1,1,"Still Up follows the exploits of Lisa, an impu...","[{'id': 132814, 'logo_path': None, 'name': 'Va...",In Production,Scripted
3,212762,The Gallows Pole,2024-12-12,[],"[{'adult': False, 'gender': 2, 'id': 126171, '...",[],1,1,Set against the backdrop of the coming industr...,"[{'id': 3324, 'logo_path': '/16fY7pucCzn7WSOYx...",In Production,Miniseries
4,212462,Mary & George,2024-12-12,[],"[{'adult': False, 'gender': 1, 'id': 1231, 'kn...",[],1,1,"The story about Mary Villiers, Countess of Buc...","[{'id': 122908, 'logo_path': None, 'name': 'He...",Planned,Miniseries
...,...,...,...,...,...,...,...,...,...,...,...,...
97,203203,Married... with Children,2023-01-01,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'adult': False, 'gender': 2, 'id': 18977, 'k...",[],1,1,An animated adaptation of the hit 1980s sitcom...,"[{'id': 11073, 'logo_path': '/aCbASRcI1MI7DXjP...",In Production,Scripted
98,202738,Greenlight – German Genius,2023-01-01,"[{'id': 35, 'name': 'Comedy'}]","[{'adult': False, 'gender': 2, 'id': 5850, 'kn...",[30],1,1,,"[{'id': 40788, 'logo_path': '/ydsYNksXNi0NzkCD...",In Production,Scripted
99,195257,15 Candles,2023-01-01,[],[],[],1,1,Four young Latinas in high school navigate fee...,"[{'id': 71293, 'logo_path': None, 'name': 'Oja...",Planned,Scripted
100,139167,Night Wanderer,2023-01-01,[],"[{'adult': False, 'gender': 1, 'id': 1006864, ...",[],1,1,"One summer night in Shanghai, Zong Ying bumps ...",[],Returning Series,Scripted


### Preprocessing the raw data to predict

In [23]:
from sklearn.preprocessing import MultiLabelBinarizer
df=preprocessing_genre(df)

In [24]:
df

,id,name,first_air_date,genre,cast_details,episode_run_time,number_of_episodes,number_of_seasons,overview,production_companies,...,Documentary,Drama,Family,Kids,Mystery,News,Reality,Sci-Fi & Fantasy,Soap,Talk
0,213569,Western,2024-12-12,[Comedy],"[{'adult': False, 'gender': 1, 'id': 1489288, ...",[],1,1,"Set in the 1800s, it follows Polly, a young hi...","[{'id': 11073, 'logo_path': '/aCbASRcI1MI7DXjP...",...,0,0,0,0,0,0,0,0,0,0
1,213338,The Buccaneers,2024-12-12,[],"[{'adult': False, 'gender': 1, 'id': 110014, '...",[],1,1,The Buccaneers adaptation follows a group of f...,[],...,0,0,0,0,0,0,0,0,0,0
2,213337,Still Up,2024-12-12,[],"[{'adult': False, 'gender': 1, 'id': 221981, '...",[],1,1,"Still Up follows the exploits of Lisa, an impu...","[{'id': 132814, 'logo_path': None, 'name': 'Va...",...,0,0,0,0,0,0,0,0,0,0
3,212762,The Gallows Pole,2024-12-12,[],"[{'adult': False, 'gender': 2, 'id': 126171, '...",[],1,1,Set against the backdrop of the coming industr...,"[{'id': 3324, 'logo_path': '/16fY7pucCzn7WSOYx...",...,0,0,0,0,0,0,0,0,0,0
4,212462,Mary & George,2024-12-12,[],"[{'adult': False, 'gender': 1, 'id': 1231, 'kn...",[],1,1,"The story about Mary Villiers, Countess of Buc...","[{'id': 122908, 'logo_path': None, 'name': 'He...",...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,203203,Married... with Children,2023-01-01,"[Animation, Comedy]","[{'adult': False, 'gender': 2, 'id': 18977, 'k...",[],1,1,An animated adaptation of the hit 1980s sitcom...,"[{'id': 11073, 'logo_path': '/aCbASRcI1MI7DXjP...",...,0,0,0,0,0,0,0,0,0,0
98,202738,Greenlight – German Genius,2023-01-01,[Comedy],"[{'adult': False, 'gender': 2, 'id': 5850, 'kn...",[30],1,1,,"[{'id': 40788, 'logo_path': '/ydsYNksXNi0NzkCD...",...,0,0,0,0,0,0,0,0,0,0
99,195257,15 Candles,2023-01-01,[],[],[],1,1,Four young Latinas in high school navigate fee...,"[{'id': 71293, 'logo_path': None, 'name': 'Oja...",...,0,0,0,0,0,0,0,0,0,0
100,139167,Night Wanderer,2023-01-01,[],"[{'adult': False, 'gender': 1, 'id': 1006864, ...",[],1,1,"One summer night in Shanghai, Zong Ying bumps ...",[],...,0,0,0,0,0,0,0,0,0,0


In [25]:
df=preprocessing_status_type(df)


In [26]:
df=preprocessing_production_companies(df)


In [27]:
df=preprocessing_episode_runtime(df)


In [28]:
df=preprocessing_cast_details(df)

In [29]:
df=preprocessing_defualt_air_date(df)


In [30]:
df=changing_genre_names(df)


In [31]:
df=popping_columns(df)


In [32]:
df=changing_prod_companynames(df)


In [33]:
df=replacing_nan_values_normalizing(df)

In [34]:
df=popping_columns_final(df)

In [35]:
df.shape

(102, 34)

In [36]:
df

,name,episode_run_time,number_of_episodes,number_of_seasons,Genre_Action & Adventure,Genre_Animation,Genre_Comedy,Genre_Crime,Genre_Documentary,Genre_Drama,...,status_In Production,status_Planned,status_Returning Series,ProdComp_Amazon Studios,ProdComp_BBC,ProdComp_Estúdios Globo,ProdComp_Sony Pictures Television Studios,ProdComp_Studio Dragon,ProdComp_Universal Television,Casts_avg_popularity
0,Western,0.0,0.055556,0.333333,0,0,1,0,0,0,...,0,1,0,1,0,0,1,0,0,0.223521
1,The Buccaneers,0.0,0.055556,0.333333,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0.498590
2,Still Up,0.0,0.055556,0.333333,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0.190067
3,The Gallows Pole,0.0,0.055556,0.333333,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0.209692
4,Mary & George,0.0,0.055556,0.333333,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0.855886
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,Married... with Children,0.0,0.055556,0.333333,0,1,1,0,0,0,...,1,0,0,0,0,0,1,0,0,1.000000
98,Greenlight – German Genius,0.333333,0.055556,0.333333,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0.167513
99,15 Candles,0.0,0.055556,0.333333,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0.000000
100,Night Wanderer,0.0,0.055556,0.333333,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0.219423


### Function for creating a final proper dataset to predict on

In [37]:
def proper_dataset(df):
    partial_cols=[]
    for col in df:
        partial_cols.append(col)
    absent_cols=[]
    all_cols=['episode_run_time', 'number_of_episodes', 'number_of_seasons', 'Genre_Action & Adventure', 'Genre_Animation', 'Genre_Comedy', 'Genre_Crime', 'Genre_Documentary', 'Genre_Drama', 'Genre_Erotic', 'Genre_Family', 'Genre_History', 'Genre_Kids', 'Genre_Music', 'Genre_Mystery', 'Genre_News', 'Genre_Reality', 'Genre_Romance', 'Genre_Sci-Fi & Fantasy', 'Genre_Soap', 'Genre_Talk', 'Genre_War & Politics', 'Genre_Western', 'type_Documentary', 'type_Miniseries', 'type_News', 'type_Reality', 'type_Scripted', 'type_Talk Show', 'type_Video', 'status_Canceled', 'status_Ended', 'status_In Production', 'status_Pilot', 'status_Planned', 'status_Returning Series', 'ProdComp_20th Century Fox Television', 'ProdComp_Amazon Studios', 'ProdComp_BBC', 'ProdComp_CBS Studios', 'ProdComp_Estúdios Globo', 'ProdComp_Madhouse', 'ProdComp_NBC', 'ProdComp_NHK', 'ProdComp_Pink Pineapple', 'ProdComp_Pol-Ka Producciones', 'ProdComp_SUNRISE', 'ProdComp_Sony Pictures Television Studios', 'ProdComp_Studio Dragon', 'ProdComp_TV Asahi', 'ProdComp_TV Tokyo', 'ProdComp_TVB', 'ProdComp_Toei Animation', 'ProdComp_Toei Company', 'ProdComp_Universal Television', 'ProdComp_Warner Bros. Television', 'Casts_avg_popularity']
    
    for each_col in all_cols:
        if each_col not in partial_cols:
            absent_cols.append(each_col)
    print(absent_cols)
    array=[0]*rows
    for an_absent_col in absent_cols:
        df[an_absent_col]=array
    return df
        

In [38]:
proper_dataset(df)

['Genre_Erotic', 'Genre_History', 'Genre_Music', 'Genre_Romance', 'Genre_War & Politics', 'Genre_Western', 'type_News', 'type_Video', 'status_Canceled', 'status_Pilot', 'ProdComp_20th Century Fox Television', 'ProdComp_CBS Studios', 'ProdComp_Madhouse', 'ProdComp_NBC', 'ProdComp_NHK', 'ProdComp_Pink Pineapple', 'ProdComp_Pol-Ka Producciones', 'ProdComp_SUNRISE', 'ProdComp_TV Asahi', 'ProdComp_TV Tokyo', 'ProdComp_TVB', 'ProdComp_Toei Animation', 'ProdComp_Toei Company', 'ProdComp_Warner Bros. Television']


,name,episode_run_time,number_of_episodes,number_of_seasons,Genre_Action & Adventure,Genre_Animation,Genre_Comedy,Genre_Crime,Genre_Documentary,Genre_Drama,...,ProdComp_NHK,ProdComp_Pink Pineapple,ProdComp_Pol-Ka Producciones,ProdComp_SUNRISE,ProdComp_TV Asahi,ProdComp_TV Tokyo,ProdComp_TVB,ProdComp_Toei Animation,ProdComp_Toei Company,ProdComp_Warner Bros. Television
0,Western,0.0,0.055556,0.333333,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,The Buccaneers,0.0,0.055556,0.333333,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Still Up,0.0,0.055556,0.333333,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Gallows Pole,0.0,0.055556,0.333333,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Mary & George,0.0,0.055556,0.333333,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,Married... with Children,0.0,0.055556,0.333333,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,Greenlight – German Genius,0.333333,0.055556,0.333333,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99,15 Candles,0.0,0.055556,0.333333,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100,Night Wanderer,0.0,0.055556,0.333333,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
df_names=df.pop('name')

In [40]:
df_names=df_names.to_frame()

### Prediction Taking Place Now

In [41]:
predictions=pickled_model.predict(df)

In [42]:
df_prediction=pd.DataFrame(predictions,columns=['Popularity'])

In [43]:
df_prediction

,Popularity
0,-6.083489e+08
1,-1.098630e+08
2,-1.774629e+08
3,-3.922710e+08
4,-3.157659e+07
...,...
97,-2.191081e+08
98,-1.824045e+08
99,-2.191081e+08
100,-1.710307e+08


In [44]:
dataframe=df_names.join(df_prediction)

In [52]:
dataframe=dataframe.rename(columns={'name':'Name'})


In [46]:
def absolute_maximum_scale(series):
        return (series / series.abs().max())*(-1)
        

In [47]:
dataframe['Predicted_Popularity']=absolute_maximum_scale(dataframe['Popularity'])

In [48]:
dataframe['Predicted_Popularity'].describe()

count    102.000000
mean       0.331097
std        0.141503
min       -0.132574
25%        0.288056
50%        0.350276
75%        0.360168
max        1.000000
Name: Predicted_Popularity, dtype: float64

In [49]:
df_popu=dataframe.pop('Popularity')


In [53]:
dataframe.head()

,Name,Predicted_Popularity
0,Western,1.000000
1,The Buccaneers,0.180592
2,Still Up,0.291712
3,The Gallows Pole,0.644813
4,Mary & George,0.051905


### Converting to CSV file 

In [51]:
dataframe.to_csv('predictions.csv')